<a href="https://colab.research.google.com/github/eeolga/article/blob/main/Formula_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np

# Load Excel data
def load_excel_data(file_path):
    """
    Load Excel data into a Pandas DataFrame.
    """
    return pd.read_excel(file_path)

# Compute Cn = WiD + WnT + WnECTS
def compute_Cn(bloom_weights, tool_weights, tool_ects):
    """
    Compute Cn based on the given formula.
    """
    Cn = bloom_weights + tool_weights + tool_ects
    return Cn

# Compute WiD = i / sum(i)
def compute_WiD(i_values):
    """
    Compute WiD for i-th Bloom level weights.
    """
    total = sum(i_values)
    return np.array(i_values) / total

# Compute WnT = (1 / sum(T)) * n
def compute_WnT(n_values):
    """
    Compute WnT for n-th tool weights.
    """
    total = sum(n_values)
    return np.array(n_values) / total

# Compute WnECTS = (3 * factor / 100) * (sum(factors))
def compute_WnECTS(factors):
    """
    Compute WnECTS for n-th tool ECTS weights.
    """
    total_factor = sum(factors)
    return (3 * np.array(factors) / 100) * total_factor

# Compute DA = WA * fA, where fA = A * A
def compute_DA(WA, A):
    """
    Compute Dynamic Assessment component DA.
    """
    fA = np.array(A) * np.array(A)
    return WA * fA

# Main function to compute Competency Indicator (CA)
def compute_CA(data):
    """
    Compute Competency Indicator (CA) using the provided formula.

    Parameters:
        data: Pandas DataFrame with required input columns.

    Returns:
        DataFrame with CA values computed.
    """
    # Compute Bloom level weights WiD
    data['WiD'] = compute_WiD(data['Bloom_level'].tolist())

    # Compute Tool weights WnT
    data['WnT'] = compute_WnT(data['Tool_competence_indication'].tolist())

    # Compute Tool ECTS WnECTS
    data['WnECTS'] = compute_WnECTS(data['Tool_ECTS'].tolist())

    # Compute Cn = WiD + WnT + WnECTS
    data['Cn'] = compute_Cn(data['WiD'], data['WnT'], data['WnECTS'])

    # Compute DA = WA * fA, where fA = A * A
    data['DA'] = compute_DA(data['Competency_Indicator_0.1'], data['Competency_Indicator_0.5'])

    # Compute CA = Cn + DA
    data['CA'] = data['Cn'] + data['DA']

    return data

# Example usage
if __name__ == "__main__":
    # Load data from an Excel file
    file_path = "/content/output 7-12-2023.xlsx"  # Replace with the actual file path
    data = load_excel_data(file_path)

    # Compute CA values
    result = compute_CA(data)

    # Save results back to an Excel file
    result.to_excel("computed_results.xlsx", index=False)
    print("Results saved to 'computed_results.xlsx'")


Results saved to 'computed_results.xlsx'
